# SKMULTILEARN

In [37]:
import numpy as np 
import pandas as pd

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import multilabel_confusion_matrix
import sklearn.metrics as skm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.multiclass import OneVsOneClassifier 
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

In [39]:
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN

In [40]:
import feather
import warnings
warnings.filterwarnings('ignore')

As seen in the EDA part the proportion of genres in the dataset is not balenced .We'll choose the model score to be the F1 score.

### Reading data from featehr format

In [41]:
# load data
url1 = "https://raw.githubusercontent.com/PicoRadia/sdataProjet/main/dataset/Spotify_train_dataset.csv"
df = pd.read_csv(url1)#31728 rows x 20 columns
# dropping duplicates
df.drop_duplicates(subset ="id", keep = False, inplace = True)

In [42]:
data = pd.read_feather('./fin')
data.head()

,index,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature__1,time_signature__3,time_signature__4,time_signature__5,mode_0,mode_1,mins,song_name_nan
0,1,0.618,0.654,-14.305,0.4220,0.226000,0.864,0.1110,0.1760,127.953,0,0,1,0,1,0,6.068300,0
1,2,0.696,0.896,-5.915,0.1080,0.016400,0.866,0.1030,0.5700,128.001,0,0,1,0,0,1,8.437500,0
2,3,0.769,0.506,-11.658,0.0452,0.002730,0.000,0.0832,0.1020,120.004,0,0,1,0,0,1,3.867433,1
3,5,0.568,0.779,-9.779,0.0323,0.000519,0.929,0.0660,0.0852,128.015,0,0,1,0,0,1,4.100150,0
4,6,0.784,0.852,-2.511,0.1660,0.001630,0.000,0.0781,0.5290,134.046,0,0,1,0,1,0,2.433400,1


In [43]:
a = df.genre
y = pd.get_dummies(a, prefix='t')

### Split to train and test data

Creating a good dataset is very important we need to have the same proportion of genres in the test set as in the training test.
=> Stratified spitting.

In [44]:
X_train, X_test, y_train, y_test = train_test_split(data, y,test_size=0.2, random_state=42, stratify=y)

In [45]:
classifier = LabelPowerset(
    classifier = RandomForestClassifier(bootstrap=True,n_jobs = -1,class_weight='balanced_subsample'),
    require_dense = [False, True]
)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
f1_score(y_test, y_pred,average='micro')

0.732619240097009

In [13]:
 print(skm.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.67      0.70      0.69       555
           1       0.77      0.78      0.77       245
           2       0.48      0.43      0.45       367
           3       0.62      0.16      0.26        61
           4       0.67      0.28      0.40       193
           5       0.52      0.48      0.50       277
           6       0.59      0.41      0.48       198
           7       0.47      0.65      0.55       640
           8       1.00      1.00      1.00       341
           9       0.90      0.91      0.91       310
          10       0.91      0.90      0.91       376
          11       0.91      0.90      0.90       289
          12       0.86      0.87      0.86       371
          13       0.88      0.91      0.89       411
          14       0.91      0.88      0.89       314

   micro avg       0.73      0.73      0.73      4948
   macro avg       0.74      0.68      0.70      4948
weighted avg       0.74   

## MLKNN